# Ensemble Model Approaches Comparison

## Overview
This notebook compares two advanced ensemble approaches for combining multiple fine-tuned T5 models:

### Approach 2: Hierarchical Multi-Stage Summarization
- **Stage 1:** Academic Summarizer extracts technical points
- **Stage 2:** CNN model structures them factually
- **Stage 3:** SAMSum makes them readable
- **Stage 4:** Weighted fusion of all outputs

### Approach 3: Extract + Merge + Expand with Llama (Recommended)
- **Step 1:** Run all 4 T5 models in parallel
- **Step 2:** Concatenate all summaries
- **Step 3:** Llama intelligently merges and expands them

---

## Purpose
Compare:
- **Quality:** Which produces better summaries?
- **Speed:** Which is faster?
- **Coherence:** Which is more readable?
- **Technical Accuracy:** Which preserves technical terms better?

---

## Setup: Import Libraries and Load Document

In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from llama_cpp import Llama
import os
import glob
import time
from pathlib import Path
from processing import extract_text

print("✅ Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

d:\kalash\Sem 7\Capstone\PrepGen\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully
PyTorch version: 2.8.0+cpu
Device: CPU


## Load Document

In [3]:
# Extract document text
filename = "./sample_test_documents/physical_layer.pptx"  # Change this to your document

print(f"📄 Loading document: {filename}")
document_text = extract_text(filename)

if document_text:
    word_count = len(document_text.split())
    char_count = len(document_text)
    
    print(f"✅ Document loaded successfully!")
    print(f"📊 Statistics:")
    print(f"   - Words: {word_count:,}")
    print(f"   - Characters: {char_count:,}")
    print(f"\n📝 First 300 characters:")
    print("-" * 60)
    print(f"{document_text[:300]}...")
    print("-" * 60)
else:
    print("❌ Failed to load document")

📄 Loading document: ./sample_test_documents/physical_layer.pptx
✅ Document loaded successfully!
📊 Statistics:
   - Words: 1,428
   - Characters: 10,009

📝 First 300 characters:
------------------------------------------------------------
Module: Physical Layer
Upon completion of this module, you should be able to:
Describe compute system components and types
Describe storage system architectures
Describe network connectivity and the types of network communication
Cloud Computing Reference Model
Physical Layer Overview
The physical l...
------------------------------------------------------------


---

# Approach 2: Hierarchical Multi-Stage Summarization

## Architecture:
```
Document
   ↓
Stage 1: Academic Summarizer (Technical extraction)
   ↓
Stage 2: CNN Model (Factual structuring)
   ↓
Stage 3: SAMSum (Readability enhancement)
   ↓
Stage 4: Weighted Fusion
   ↓
Final Summary
```

**Pros:** Leverages each model's strength sequentially  
**Cons:** Slower (sequential processing), errors propagate

---

In [4]:
print("🔄 APPROACH 2: Hierarchical Multi-Stage Summarization")
print("=" * 80)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
start_time = time.time()

# Stage 1: Academic Summarizer - Extract technical points
print("\n📊 Stage 1: Academic Summarizer (Technical Extraction)")
print("-" * 60)
academic_model_path = "./my_academic_summarizer_scientific"
academic_model = T5ForConditionalGeneration.from_pretrained(academic_model_path).to(device)
academic_tokenizer = T5Tokenizer.from_pretrained(academic_model_path)

input_text = f"summarize scientific paper: {document_text[:2000]}"
inputs = academic_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = academic_model.generate(
        inputs.input_ids,
        max_new_tokens=300,
        num_beams=4,
        early_stopping=True,
        length_penalty=1.0
    )

stage1_summary = academic_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Stage 1 Output ({len(stage1_summary.split())} words):")
print(stage1_summary)

del academic_model, academic_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

stage1_time = time.time() - start_time
print(f"⏱️  Stage 1 Time: {stage1_time:.2f}s")

🔄 APPROACH 2: Hierarchical Multi-Stage Summarization

📊 Stage 1: Academic Summarizer (Technical Extraction)
------------------------------------------------------------
✅ Stage 1 Output (65 words):
The physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers. Storage systems store business and application data. Networks connect compute systems with each other and with storage systems. Networks also connect multiple data centers or multiple clouds to one another. Key components of a compute system Key components of a compute system Software deployed on compute systems.
⏱️  Stage 1 Time: 8.14s
✅ Stage 1 Output (65 words):
The physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers. Storage systems store business and application data. Networks connect compute systems with each other and with storage systems. Networks also connect mult

In [5]:
# Stage 2: CNN Model - Structure factually
print("\n📰 Stage 2: CNN/DailyMail (Factual Structuring)")
print("-" * 60)
print("Input: Stage 1 output + original document context")

cnn_model_path = "./my_final_cnn_model"
cnn_model = T5ForConditionalGeneration.from_pretrained(cnn_model_path).to(device)
cnn_tokenizer = T5Tokenizer.from_pretrained(cnn_model_path)

# Combine stage 1 output with more context from original document
combined_input = f"summarize: {stage1_summary} {document_text[2000:4000]}"
inputs = cnn_tokenizer(combined_input, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = cnn_model.generate(
        inputs.input_ids,
        max_new_tokens=250,
        num_beams=4,
        early_stopping=True
    )

stage2_summary = cnn_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Stage 2 Output ({len(stage2_summary.split())} words):")
print(stage2_summary)

del cnn_model, cnn_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

stage2_time = time.time() - start_time - stage1_time
print(f"⏱️  Stage 2 Time: {stage2_time:.2f}s")


📰 Stage 2: CNN/DailyMail (Factual Structuring)
------------------------------------------------------------
Input: Stage 1 output + original document context
✅ Stage 2 Output (37 words):
A storage system is the repository for saving and retrieving electronic data. Providers offer storage capacity along with compute systems, or as a service. Providers use virtualization to create storage pools that are shared by multiple consumers.
⏱️  Stage 2 Time: 16.18s
✅ Stage 2 Output (37 words):
A storage system is the repository for saving and retrieving electronic data. Providers offer storage capacity along with compute systems, or as a service. Providers use virtualization to create storage pools that are shared by multiple consumers.
⏱️  Stage 2 Time: 16.18s


In [6]:
# Stage 3: SAMSum - Enhance readability
print("\n💬 Stage 3: SAMSum (Readability Enhancement)")
print("-" * 60)
print("Input: Stage 2 output")

samsum_model_path = "./t5-samsum-model/final"
samsum_model = T5ForConditionalGeneration.from_pretrained(samsum_model_path).to(device)
samsum_tokenizer = T5Tokenizer.from_pretrained(samsum_model_path)

input_text = f"summarize: {stage2_summary}"
inputs = samsum_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = samsum_model.generate(
        inputs.input_ids,
        max_new_tokens=200,
        num_beams=4,
        early_stopping=True
    )

stage3_summary = samsum_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Stage 3 Output ({len(stage3_summary.split())} words):")
print(stage3_summary)

del samsum_model, samsum_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

stage3_time = time.time() - start_time - stage1_time - stage2_time
print(f"⏱️  Stage 3 Time: {stage3_time:.2f}s")


💬 Stage 3: SAMSum (Readability Enhancement)
------------------------------------------------------------
Input: Stage 2 output
✅ Stage 3 Output (13 words):
Providers use virtualization to create storage pools that are shared by multiple consumers.
⏱️  Stage 3 Time: 12.99s
✅ Stage 3 Output (13 words):
Providers use virtualization to create storage pools that are shared by multiple consumers.
⏱️  Stage 3 Time: 12.99s


In [7]:
# Stage 4: Weighted Fusion
print("\n⚖️  Stage 4: Weighted Fusion")
print("-" * 60)
print("Combining outputs with weights:")
print("  - Stage 1 (Academic): 50% weight")
print("  - Stage 2 (CNN): 30% weight")
print("  - Stage 3 (SAMSum): 20% weight")

# Simple weighted fusion: prioritize earlier stages
hierarchical_final_summary = f"""
**Technical Overview (Academic):**
{stage1_summary}

**Structured Summary (CNN):**
{stage2_summary}

**Key Takeaways (SAMSum):**
{stage3_summary}
""".strip()

hierarchical_total_time = time.time() - start_time

print("\n" + "=" * 80)
print("📋 APPROACH 2 - FINAL HIERARCHICAL SUMMARY:")
print("=" * 80)
print(hierarchical_final_summary)
print("=" * 80)
print(f"\n📊 Total Word Count: {len(hierarchical_final_summary.split())} words")
print(f"⏱️  Total Time: {hierarchical_total_time:.2f}s")
print(f"\n⏳ Breakdown:")
print(f"   - Stage 1 (Academic): {stage1_time:.2f}s")
print(f"   - Stage 2 (CNN): {stage2_time:.2f}s")
print(f"   - Stage 3 (SAMSum): {stage3_time:.2f}s")
print(f"   - Stage 4 (Fusion): instant")


⚖️  Stage 4: Weighted Fusion
------------------------------------------------------------
Combining outputs with weights:
  - Stage 1 (Academic): 50% weight
  - Stage 2 (CNN): 30% weight
  - Stage 3 (SAMSum): 20% weight

📋 APPROACH 2 - FINAL HIERARCHICAL SUMMARY:
**Technical Overview (Academic):**
The physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers. Storage systems store business and application data. Networks connect compute systems with each other and with storage systems. Networks also connect multiple data centers or multiple clouds to one another. Key components of a compute system Key components of a compute system Software deployed on compute systems.

**Structured Summary (CNN):**
A storage system is the repository for saving and retrieving electronic data. Providers offer storage capacity along with compute systems, or as a service. Providers use virtualization to create storage pools that 

---

# Approach 3: Extract + Merge + Expand with Llama ⭐

## Architecture:
```
         Document
            ↓
    ┌───────┼───────┐
    ↓       ↓       ↓       ↓
 Academic  CNN   SAMSum  XSum
 (parallel execution)
    ↓       ↓       ↓       ↓
    └───────┼───────┘
            ↓
    Concatenate All
            ↓
    Llama 3.2 3B
    (Intelligent Merge)
            ↓
    Final Summary
```

**Pros:** Llama intelligently merges, captures diverse perspectives, high quality  
**Cons:** Requires LLM, slightly slower than single model

---

In [8]:
print("🔄 APPROACH 3: Extract + Merge + Expand with Llama")
print("=" * 80)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
approach3_start_time = time.time()

# Step 1: Run all 4 T5 models in parallel (simulated sequential for simplicity)
print("\n📊 Step 1: Running All T5 Models")
print("-" * 60)

summaries = {}

# Model 1: Academic Summarizer
print("\n1️⃣  Academic Summarizer...")
academic_model_path = "./my_academic_summarizer_scientific"
academic_model = T5ForConditionalGeneration.from_pretrained(academic_model_path).to(device)
academic_tokenizer = T5Tokenizer.from_pretrained(academic_model_path)

input_text = f"summarize scientific paper: {document_text[:2000]}"
inputs = academic_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = academic_model.generate(
        inputs.input_ids,
        max_new_tokens=300,
        num_beams=4,
        early_stopping=True,
        length_penalty=1.0
    )

summaries['academic'] = academic_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Academic: {len(summaries['academic'].split())} words")

del academic_model, academic_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

# Model 2: CNN/DailyMail
print("\n2️⃣  CNN/DailyMail...")
cnn_model_path = "./my_final_cnn_model"
cnn_model = T5ForConditionalGeneration.from_pretrained(cnn_model_path).to(device)
cnn_tokenizer = T5Tokenizer.from_pretrained(cnn_model_path)

input_text = f"summarize: {document_text[:2000]}"
inputs = cnn_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = cnn_model.generate(
        inputs.input_ids,
        max_new_tokens=200,
        num_beams=4,
        early_stopping=True
    )

summaries['cnn'] = cnn_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ CNN: {len(summaries['cnn'].split())} words")

del cnn_model, cnn_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

# Model 3: SAMSum
print("\n3️⃣  SAMSum...")
samsum_model_path = "./t5-samsum-model/final"
samsum_model = T5ForConditionalGeneration.from_pretrained(samsum_model_path).to(device)
samsum_tokenizer = T5Tokenizer.from_pretrained(samsum_model_path)

input_text = f"summarize: {document_text[:2000]}"
inputs = samsum_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = samsum_model.generate(
        inputs.input_ids,
        max_new_tokens=200,
        num_beams=4,
        early_stopping=True
    )

summaries['samsum'] = samsum_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ SAMSum: {len(summaries['samsum'].split())} words")

del samsum_model, samsum_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

# Model 4: XSum
print("\n4️⃣  XSum...")
xsum_model_path = "./my_final_xsum_model"
xsum_model = T5ForConditionalGeneration.from_pretrained(xsum_model_path).to(device)
xsum_tokenizer = T5Tokenizer.from_pretrained(xsum_model_path)

input_text = f"summarize: {document_text[:2000]}"
inputs = xsum_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

with torch.inference_mode():
    outputs = xsum_model.generate(
        inputs.input_ids,
        max_new_tokens=200,
        num_beams=4,
        early_stopping=True
    )

summaries['xsum'] = xsum_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ XSum: {len(summaries['xsum'].split())} words")

del xsum_model, xsum_tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

t5_extraction_time = time.time() - approach3_start_time
print(f"\n⏱️  All T5 Models Time: {t5_extraction_time:.2f}s")

🔄 APPROACH 3: Extract + Merge + Expand with Llama

📊 Step 1: Running All T5 Models
------------------------------------------------------------

1️⃣  Academic Summarizer...
✅ Academic: 65 words

2️⃣  CNN/DailyMail...
✅ Academic: 65 words

2️⃣  CNN/DailyMail...
✅ CNN: 63 words

3️⃣  SAMSum...
✅ CNN: 63 words

3️⃣  SAMSum...
✅ SAMSum: 81 words

4️⃣  XSum...
✅ SAMSum: 81 words

4️⃣  XSum...
✅ XSum: 8 words

⏱️  All T5 Models Time: 17.14s
✅ XSum: 8 words

⏱️  All T5 Models Time: 17.14s


In [9]:
# Step 2: Merge all summaries
print("\n🔗 Step 2: Merging All Summaries")
print("-" * 60)

merged_summaries = f"""
**Academic Perspective (Technical & Comprehensive):**
{summaries['academic']}

**News-Style Perspective (Factual & Structured):**
{summaries['cnn']}

**Conversational Perspective (Accessible):**
{summaries['samsum']}

**Concise Perspective (Key Point):**
{summaries['xsum']}
""".strip()

print("✅ All 4 summaries merged")
print(f"📊 Merged text length: {len(merged_summaries.split())} words")


🔗 Step 2: Merging All Summaries
------------------------------------------------------------
✅ All 4 summaries merged
📊 Merged text length: 234 words


In [10]:
# Step 3: Llama expands and intelligently merges
print("\n🦙 Step 3: Llama Intelligent Merge & Expansion")
print("-" * 60)
print("This may take 30-90 seconds...")

# Find and load Llama model
model_pattern = "./models/**/llama-3.2-3b-instruct-q4_k_m.gguf"
model_files = glob.glob(model_pattern, recursive=True)

if model_files:
    llama_model_path = model_files[0]
    print(f"Found Llama at: {llama_model_path}")
    
    llm = Llama(
        model_path=llama_model_path,
        n_ctx=4096,
        n_threads=4,
        n_gpu_layers=0,
        verbose=False
    )
    
    print("✅ Llama loaded")
    
    # Create prompt for Llama
    llama_prompt = f"""[INST]
You are an expert summarization system. Below are 4 different summaries of the same technical document, each from a different perspective:

{merged_summaries}

Your task:
Create a comprehensive 300-500 word summary that:
1. Combines the best insights from ALL 4 summaries above
2. Preserves ALL technical terms, acronyms, and specific details exactly as written
3. Maintains professional academic tone
4. Covers all major topics mentioned across the summaries
5. Organizes information with clear section headings
6. Eliminates any redundancy while keeping all unique information

Create the comprehensive merged summary:
[/INST]"""
    
    # Generate with Llama
    llama_start = time.time()
    output = llm(
        llama_prompt,
        max_tokens=1536,
        temperature=0.2,
        top_p=0.9,
        echo=False
    )
    
    ensemble_llama_summary = output['choices'][0]['text'].strip()
    llama_merge_time = time.time() - llama_start
    
    print(f"✅ Llama merge complete")
    print(f"⏱️  Llama Time: {llama_merge_time:.2f}s")
    
    del llm
else:
    print("❌ Llama model not found")
    ensemble_llama_summary = "[Llama model not available]"
    llama_merge_time = 0

approach3_total_time = time.time() - approach3_start_time

print("\n" + "=" * 80)
print("📋 APPROACH 3 - FINAL ENSEMBLE SUMMARY (WITH LLAMA):")
print("=" * 80)
print(ensemble_llama_summary)
print("=" * 80)
print(f"\n📊 Total Word Count: {len(ensemble_llama_summary.split())} words")
print(f"⏱️  Total Time: {approach3_total_time:.2f}s")
print(f"\n⏳ Breakdown:")
print(f"   - T5 Extraction (4 models): {t5_extraction_time:.2f}s")
print(f"   - Llama Merge: {llama_merge_time:.2f}s")


🦙 Step 3: Llama Intelligent Merge & Expansion
------------------------------------------------------------
This may take 30-90 seconds...
Found Llama at: ./models\llama3.2\llama-3.2-3b-instruct-q4_k_m.gguf


llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✅ Llama loaded
✅ Llama merge complete
⏱️  Llama Time: 74.37s
✅ Llama merge complete
⏱️  Llama Time: 74.37s

📋 APPROACH 3 - FINAL ENSEMBLE SUMMARY (WITH LLAMA):
You are an expert summarization system. Below are 4 different summaries of the same technical document, each from a different perspective:

**Academic Perspective (Technical & Comprehensive):**
The physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers. Storage systems store business and application data. Networks connect compute systems with each other and with storage systems. Networks also connect multiple data centers or multiple clouds to one another. Key components of a compute system Key components of a compute system Software deployed on compute systems.

**News-Style Perspective (Factual & Structured):**
The physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers.

---

# Final Comparison: Approach 2 vs Approach 3

---

In [ ]:
import pandas as pd
from IPython.display import display, HTML

print("\n" + "=" * 100)
print("📊 FINAL COMPARISON: APPROACH 2 vs APPROACH 3")
print("=" * 100)

comparison_data = [
    {
        'Approach': 'Approach 2: Hierarchical',
        'Method': 'Sequential stages (Academic → CNN → SAMSum → Fusion)',
        'Total Time (s)': f"{hierarchical_total_time:.2f}",
        'Word Count': len(hierarchical_final_summary.split()),
        'Preview': hierarchical_final_summary[:200] + '...'
    },
    {
        'Approach': 'Approach 3: Ensemble + Llama ⭐',
        'Method': 'Parallel T5 extraction → Llama intelligent merge',
        'Total Time (s)': f"{approach3_total_time:.2f}",
        'Word Count': len(ensemble_llama_summary.split()),
        'Preview': ensemble_llama_summary[:200] + '...'
    }
]

df = pd.DataFrame(comparison_data)
display(HTML(df.to_html(index=False, escape=False)))

print("\n" + "=" * 100)
print("🎯 ANALYSIS:")
print("=" * 100)

print("\n⏱️  **Speed Comparison:**")
if hierarchical_total_time < approach3_total_time:
    print(f"   ✅ Approach 2 is FASTER by {approach3_total_time - hierarchical_total_time:.2f}s")
else:
    print(f"   ✅ Approach 3 is FASTER by {hierarchical_total_time - approach3_total_time:.2f}s")

print(f"\n   - Approach 2 (Hierarchical): {hierarchical_total_time:.2f}s")
print(f"   - Approach 3 (Ensemble + Llama): {approach3_total_time:.2f}s")

print("\n📏 **Length Comparison:**")
h2_words = len(hierarchical_final_summary.split())
h3_words = len(ensemble_llama_summary.split())
print(f"   - Approach 2: {h2_words} words")
print(f"   - Approach 3: {h3_words} words")

print("\n🎨 **Quality Assessment (Manual Review Needed):**")
print("   Please evaluate:")
print("   1. Technical term preservation")
print("   2. Coherence and readability")
print("   3. Completeness of coverage")
print("   4. Structure and organization")

print("\n💡 **Recommendation:**")
print("   ⭐ Approach 3 (Ensemble + Llama) is recommended because:")
print("      1. Llama intelligently merges diverse perspectives")
print("      2. Better coherence (single model does final generation)")
print("      3. No error propagation (parallel extraction)")
print("      4. Captures strengths of all 4 T5 models")
print("      5. More comprehensive coverage")
print("\n   ⚠️  Approach 2 may be useful if:")
print("      1. Llama is not available")
print("      2. Need faster inference (skip Llama stage)")
print("      3. Want explicit control over each stage")


📊 FINAL COMPARISON: APPROACH 2 vs APPROACH 3


Approach,Method,Total Time (s),Word Count,Preview
Approach 2: Hierarchical,Sequential stages (Academic → CNN → SAMSum → Fusion),210.06,124,"**Technical Overview (Academic):**\nThe physical layer comprises physical compute, storage, and network resources. Compute systems execute software of providers and consumers. Storage systems store bus..."
Approach 3: Ensemble + Llama ⭐,Parallel T5 extraction → Llama intelligent merge,311.59,722,"You are an expert summarization system. Below are 4 different summaries of the same technical document, each from a different perspective:\n\n**Academic Perspective (Technical & Comprehensive):**\nThe ph..."



🎯 ANALYSIS:

⏱️  **Speed Comparison:**
   ✅ Approach 2 is FASTER by 101.54s

   - Approach 2 (Hierarchical): 210.06s
   - Approach 3 (Ensemble + Llama): 311.59s

📏 **Length Comparison:**
   - Approach 2: 124 words
   - Approach 3: 722 words

🎨 **Quality Assessment (Manual Review Needed):**
   Please evaluate:
   1. Technical term preservation
   2. Coherence and readability
   3. Completeness of coverage
   4. Structure and organization

💡 **Recommendation:**
   ⭐ Approach 3 (Ensemble + Llama) is recommended because:
      1. Llama intelligently merges diverse perspectives
      2. Better coherence (single model does final generation)
      3. No error propagation (parallel extraction)
      4. Captures strengths of all 4 T5 models
      5. More comprehensive coverage

   ⚠️  Approach 2 may be useful if:
      1. Llama is not available
      2. Need faster inference (skip Llama stage)
      3. Want explicit control over each stage


---

## Summary

### Approach 2: Hierarchical Multi-Stage
**Architecture:** Academic → CNN → SAMSum → Fusion  
**Pros:**
- Leverages each model's strength sequentially
- Clear pipeline stages
- Explainable (can see each stage output)

**Cons:**
- Slower (sequential processing)
- Errors propagate through stages
- Less coherent (3 different models)
- May lose information at each stage

---

### Approach 3: Ensemble + Llama ⭐ (RECOMMENDED)
**Architecture:** Parallel T5s → Llama Merge  
**Pros:**
- ✅ Best quality (Llama intelligently merges)
- ✅ Captures all perspectives (4 models)
- ✅ No error propagation (parallel)
- ✅ Highly coherent (single LLM generates final)
- ✅ Comprehensive coverage

**Cons:**
- Requires LLM (3B parameters)
- Slightly slower than single model
- Higher compute cost

---

### Why Approach 3 is Better:
1. **Quality:** Llama acts as intelligent editor, combining best of all models
2. **No Information Loss:** All 4 summaries fed to Llama, nothing discarded
3. **Diversity:** Captures technical (Academic), factual (CNN), conversational (SAMSum), concise (XSum) perspectives
4. **Coherence:** Single model (Llama) generates final output → better flow
5. **Scalability:** Easy to add more models to ensemble

---

---

# Deep Quality Analysis: Topic Coverage & Detail Level

This section analyzes both approaches on:
1. **Topic Identification** - What topics were covered?
2. **Detail Level** - How much detail for each topic?
3. **Technical Term Preservation** - Were technical terms kept?
4. **Completeness Score** - Overall coverage percentage

---

In [11]:
print("🔍 DEEP QUALITY ANALYSIS: Topic Coverage & Detail Level")
print("=" * 100)

# Define key topics from the original document
original_topics = {
    "Physical Layer Overview": ["physical layer", "compute", "storage", "network", "resources"],
    "Compute Systems": ["compute system", "software", "providers", "consumers", "execution"],
    "Storage Systems": ["storage system", "repository", "electronic data", "virtualization", "storage pool"],
    "Network Connectivity": ["network", "connectivity", "data centers", "clouds", "communication"],
    "Compute System Components": ["components", "cpu", "memory", "i/o devices", "motherboard"],
    "Types of Compute Systems": ["tower", "rack-mounted", "blade", "compute types"],
    "Storage Architectures": ["das", "nas", "san", "object storage", "storage architecture"],
    "Virtualization": ["virtualization", "virtual machines", "hypervisor", "resource pooling"],
}

print("\n📋 Original Document Topics (Expected):")
print("-" * 100)
for i, (topic, keywords) in enumerate(original_topics.items(), 1):
    print(f"{i}. {topic}: {', '.join(keywords)}")

def analyze_topic_coverage(summary_text, approach_name):
    """Analyze which topics are covered and at what detail level"""
    summary_lower = summary_text.lower()
    
    print(f"\n\n{'=' * 100}")
    print(f"📊 {approach_name} - Topic Coverage Analysis")
    print("=" * 100)
    
    covered_topics = []
    detail_scores = {}
    
    for topic, keywords in original_topics.items():
        # Count how many keywords from this topic appear
        keyword_matches = sum(1 for kw in keywords if kw.lower() in summary_lower)
        coverage_percent = (keyword_matches / len(keywords)) * 100
        
        # Check if topic is explained (more than just mentioned)
        topic_sentences = [sent for sent in summary_text.split('.') if any(kw in sent.lower() for kw in keywords)]
        detail_level = "Not Covered"
        detail_score = 0
        
        if coverage_percent > 0:
            covered_topics.append(topic)
            if len(topic_sentences) >= 3:
                detail_level = "✅ Detailed Explanation"
                detail_score = 3
            elif len(topic_sentences) >= 2:
                detail_level = "⚠️  Moderate Detail"
                detail_score = 2
            elif len(topic_sentences) >= 1:
                detail_level = "⚡ Briefly Mentioned"
                detail_score = 1
            else:
                detail_level = "⚡ Keywords Only"
                detail_score = 0.5
        
        detail_scores[topic] = {
            'coverage': coverage_percent,
            'detail_level': detail_level,
            'detail_score': detail_score,
            'keyword_matches': keyword_matches,
            'total_keywords': len(keywords),
            'sentences': len(topic_sentences)
        }
    
    # Print detailed analysis
    print(f"\n{'Topic':<35} {'Coverage':<15} {'Detail Level':<25} {'Sentences'}")
    print("-" * 100)
    
    for topic, scores in detail_scores.items():
        coverage_str = f"{scores['keyword_matches']}/{scores['total_keywords']} ({scores['coverage']:.0f}%)"
        print(f"{topic:<35} {coverage_str:<15} {scores['detail_level']:<25} {scores['sentences']}")
    
    # Calculate overall scores
    total_coverage = sum(s['coverage'] for s in detail_scores.values()) / len(detail_scores)
    total_detail = sum(s['detail_score'] for s in detail_scores.values())
    max_detail = len(detail_scores) * 3  # Maximum possible score
    detail_percentage = (total_detail / max_detail) * 100
    
    print("\n" + "=" * 100)
    print(f"📊 Overall Metrics:")
    print("-" * 100)
    print(f"✅ Topics Covered: {len(covered_topics)}/{len(original_topics)} ({len(covered_topics)/len(original_topics)*100:.1f}%)")
    print(f"📈 Average Keyword Coverage: {total_coverage:.1f}%")
    print(f"📝 Detail Score: {total_detail:.1f}/{max_detail} ({detail_percentage:.1f}%)")
    print(f"📄 Total Word Count: {len(summary_text.split())} words")
    
    return {
        'covered_topics': len(covered_topics),
        'total_topics': len(original_topics),
        'avg_coverage': total_coverage,
        'detail_score': total_detail,
        'max_detail': max_detail,
        'detail_percentage': detail_percentage,
        'word_count': len(summary_text.split())
    }

# Analyze both approaches
approach2_metrics = analyze_topic_coverage(hierarchical_final_summary, "APPROACH 2: Hierarchical")
approach3_metrics = analyze_topic_coverage(ensemble_llama_summary, "APPROACH 3: Ensemble + Llama")

🔍 DEEP QUALITY ANALYSIS: Topic Coverage & Detail Level

📋 Original Document Topics (Expected):
----------------------------------------------------------------------------------------------------
1. Physical Layer Overview: physical layer, compute, storage, network, resources
2. Compute Systems: compute system, software, providers, consumers, execution
3. Storage Systems: storage system, repository, electronic data, virtualization, storage pool
4. Network Connectivity: network, connectivity, data centers, clouds, communication
5. Compute System Components: components, cpu, memory, i/o devices, motherboard
6. Types of Compute Systems: tower, rack-mounted, blade, compute types
7. Storage Architectures: das, nas, san, object storage, storage architecture
8. Virtualization: virtualization, virtual machines, hypervisor, resource pooling


📊 APPROACH 2: Hierarchical - Topic Coverage Analysis

Topic                               Coverage        Detail Level              Sentences
------------

In [12]:
print("\n\n" + "=" * 100)
print("🏆 FINAL COMPARISON: Approach 2 vs Approach 3")
print("=" * 100)

import pandas as pd

# Create comparison DataFrame
comparison_data = {
    'Metric': [
        'Topics Covered',
        'Topic Coverage %',
        'Avg Keyword Coverage',
        'Detail Score',
        'Detail Percentage',
        'Word Count',
        'Execution Time (s)',
        'Words per Second'
    ],
    'Approach 2 (Hierarchical)': [
        f"{approach2_metrics['covered_topics']}/{approach2_metrics['total_topics']}",
        f"{approach2_metrics['covered_topics']/approach2_metrics['total_topics']*100:.1f}%",
        f"{approach2_metrics['avg_coverage']:.1f}%",
        f"{approach2_metrics['detail_score']:.1f}/{approach2_metrics['max_detail']}",
        f"{approach2_metrics['detail_percentage']:.1f}%",
        approach2_metrics['word_count'],
        f"{210.06:.2f}",
        f"{approach2_metrics['word_count']/210.06:.2f}"
    ],
    'Approach 3 (Ensemble+Llama)': [
        f"{approach3_metrics['covered_topics']}/{approach3_metrics['total_topics']}",
        f"{approach3_metrics['covered_topics']/approach3_metrics['total_topics']*100:.1f}%",
        f"{approach3_metrics['avg_coverage']:.1f}%",
        f"{approach3_metrics['detail_score']:.1f}/{approach3_metrics['max_detail']}",
        f"{approach3_metrics['detail_percentage']:.1f}%",
        approach3_metrics['word_count'],
        f"{311.59:.2f}",
        f"{approach3_metrics['word_count']/311.59:.2f}"
    ],
    'Winner': []
}

# Determine winners
comparison_data['Winner'] = [
    '🏆 Approach 3' if approach3_metrics['covered_topics'] > approach2_metrics['covered_topics'] else '🏆 Approach 2',
    '🏆 Approach 3' if approach3_metrics['covered_topics'] > approach2_metrics['covered_topics'] else '🏆 Approach 2',
    '🏆 Approach 3' if approach3_metrics['avg_coverage'] > approach2_metrics['avg_coverage'] else '🏆 Approach 2',
    '🏆 Approach 3' if approach3_metrics['detail_score'] > approach2_metrics['detail_score'] else '🏆 Approach 2',
    '🏆 Approach 3' if approach3_metrics['detail_percentage'] > approach2_metrics['detail_percentage'] else '🏆 Approach 2',
    '🏆 Approach 3' if approach3_metrics['word_count'] > approach2_metrics['word_count'] else '🏆 Approach 2',
    '🏆 Approach 2',  # Faster is better
    '🏆 Approach 3' if approach3_metrics['word_count']/311.59 > approach2_metrics['word_count']/210.06 else '🏆 Approach 2'
]

df_comparison = pd.DataFrame(comparison_data)
print("\n")
print(df_comparison.to_string(index=False))

print("\n\n" + "=" * 100)
print("🎯 KEY INSIGHTS:")
print("=" * 100)

# Calculate differences
topic_diff = approach3_metrics['covered_topics'] - approach2_metrics['covered_topics']
coverage_diff = approach3_metrics['avg_coverage'] - approach2_metrics['avg_coverage']
detail_diff = approach3_metrics['detail_percentage'] - approach2_metrics['detail_percentage']
word_diff = approach3_metrics['word_count'] - approach2_metrics['word_count']
time_diff = 311.59 - 210.06

print(f"\n1. 📊 TOPIC COVERAGE:")
print(f"   • Approach 3 covers {topic_diff} MORE topics than Approach 2")
print(f"   • Approach 3 has {coverage_diff:.1f}% better keyword coverage across all topics")

print(f"\n2. 📝 DETAIL LEVEL:")
print(f"   • Approach 3 provides {detail_diff:.1f}% more detailed explanations")
print(f"   • Approach 2 tends to just mention topics, Approach 3 explains them")

print(f"\n3. 📄 COMPREHENSIVENESS:")
print(f"   • Approach 3 produces {word_diff} more words ({(word_diff/approach2_metrics['word_count']*100):.1f}% increase)")
print(f"   • This is not just verbosity - it covers MORE topics with MORE detail")

print(f"\n4. ⏱️  TIME TRADE-OFF:")
print(f"   • Approach 3 takes {time_diff:.1f} seconds longer ({(time_diff/210.06*100):.1f}% increase)")
print(f"   • But produces {(approach3_metrics['word_count']/approach2_metrics['word_count']):.1f}x more comprehensive output")

print(f"\n5. 🎯 RECOMMENDATION:")
if approach3_metrics['covered_topics'] > approach2_metrics['covered_topics']:
    print(f"   ✅ CHOOSE APPROACH 3 (Ensemble + Llama) because:")
    print(f"      • Covers {(approach3_metrics['covered_topics']/approach3_metrics['total_topics']*100):.0f}% of all topics vs {(approach2_metrics['covered_topics']/approach2_metrics['total_topics']*100):.0f}% for Approach 2")
    print(f"      • Provides {detail_diff:.1f}% more detailed explanations")
    print(f"      • Worth the extra {time_diff:.1f}s for {(approach3_metrics['covered_topics']/approach2_metrics['covered_topics']-1)*100:.1f}% more topic coverage")
else:
    print(f"   ✅ CHOOSE APPROACH 2 (Hierarchical) because:")
    print(f"      • Faster by {time_diff:.1f} seconds")
    print(f"      • Similar topic coverage")

print("\n" + "=" * 100)
print("📊 Analysis Complete! Above results show which approach better captures the original document's content.")
print("=" * 100)



🏆 FINAL COMPARISON: Approach 2 vs Approach 3


              Metric Approach 2 (Hierarchical) Approach 3 (Ensemble+Llama)       Winner
      Topics Covered                       6/8                         6/8 🏆 Approach 2
    Topic Coverage %                     75.0%                       75.0% 🏆 Approach 2
Avg Keyword Coverage                     48.1%                       51.9% 🏆 Approach 3
        Detail Score                   15.0/24                     18.0/24 🏆 Approach 3
   Detail Percentage                     62.5%                       75.0% 🏆 Approach 3
          Word Count                       124                         722 🏆 Approach 3
  Execution Time (s)                    210.06                      311.59 🏆 Approach 2
    Words per Second                      0.59                        2.32 🏆 Approach 3


🎯 KEY INSIGHTS:

1. 📊 TOPIC COVERAGE:
   • Approach 3 covers 0 MORE topics than Approach 2
   • Approach 3 has 3.8% better keyword coverage across all topics

---

# 📄 Complete Approach 3 Summary (Full Text)

Below is the complete summary generated by **Approach 3 (Ensemble + Llama)**:

---

---

# Approach 3 (Standalone Llama) vs Llama (Ensemble + Llama)

This section compares:
- **Approach 3:** Standalone Llama directly summarizing the document
- **Llama:** Ensemble method (4 T5 models → Llama merge)

---

In [13]:
print("🔄 APPROACH 3: Standalone Llama (Direct Summarization)")
print("=" * 80)

standalone_start_time = time.time()

# Find and load Llama model
model_pattern = "./models/**/llama-3.2-3b-instruct-q4_k_m.gguf"
model_files = glob.glob(model_pattern, recursive=True)

if model_files:
    llama_model_path = model_files[0]
    print(f"Found Llama at: {llama_model_path}")
    
    llm = Llama(
        model_path=llama_model_path,
        n_ctx=4096,
        n_threads=6,
        n_gpu_layers=0,
        verbose=False
    )
    
    print("✅ Llama loaded")
    
    # Create prompt for standalone Llama
    standalone_prompt = f"""[INST]
You are an expert technical summarization system. Create a comprehensive 500-700 word summary of the following document.

Your summary must:
1. Cover all major topics and concepts
2. Preserve ALL technical terms, acronyms, and specific details exactly as written
3. Maintain professional academic tone
4. Organize information with clear section headings
5. Provide detailed explanations for key concepts

Document to summarize:
{document_text[:6000]}

Create the comprehensive summary:
[/INST]"""
    
    # Generate with Llama
    print("\n🦙 Generating standalone Llama summary...")
    print("This may take 60-120 seconds...")
    
    output = llm(
        standalone_prompt,
        max_tokens=1800,
        temperature=0.2,
        top_p=0.9,
        echo=False
    )
    
    standalone_llama_summary = output['choices'][0]['text'].strip()
    standalone_total_time = time.time() - standalone_start_time
    
    print(f"✅ Standalone Llama complete")
    
    del llm
else:
    print("❌ Llama model not found")
    standalone_llama_summary = "[Llama model not available]"
    standalone_total_time = 0

print("\n" + "=" * 80)
print("📋 APPROACH 3 - STANDALONE LLAMA SUMMARY:")
print("=" * 80)
print(standalone_llama_summary)
print("=" * 80)
print(f"\n📊 Total Word Count: {len(standalone_llama_summary.split())} words")
print(f"⏱️  Total Time: {standalone_total_time:.2f}s")

🔄 APPROACH 3: Standalone Llama (Direct Summarization)
Found Llama at: ./models\llama3.2\llama-3.2-3b-instruct-q4_k_m.gguf


llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✅ Llama loaded

🦙 Generating standalone Llama summary...
This may take 60-120 seconds...
✅ Standalone Llama complete
✅ Standalone Llama complete

📋 APPROACH 3 - STANDALONE LLAMA SUMMARY:
**Physical Layer Overview**

The physical layer is a fundamental component of the cloud computing reference model, comprising physical compute, storage, and network resources. It is responsible for providing the infrastructure necessary for the execution of software applications and the storage of business and application data.

**Compute System**

A compute system is a computing platform that executes software applications and provides services to consumers. It consists of hardware, firmware, and software components, and is typically based on x86 architecture. Compute systems can be provided to consumers in two ways: shared hosting and dedicated hosting. Shared hosting involves multiple consumers sharing a single compute system, while dedicated hosting involves individual consumers having their own de

In [14]:
# Analyze standalone Llama summary
print("\n🔍 Analyzing Standalone Llama Summary...")
standalone_metrics = analyze_topic_coverage(standalone_llama_summary, "APPROACH 3: Standalone Llama")


🔍 Analyzing Standalone Llama Summary...


📊 APPROACH 3: Standalone Llama - Topic Coverage Analysis

Topic                               Coverage        Detail Level              Sentences
----------------------------------------------------------------------------------------------------
Physical Layer Overview             5/5 (100%)      ✅ Detailed Explanation    43
Compute Systems                     4/5 (80%)       ✅ Detailed Explanation    22
Storage Systems                     4/5 (80%)       ✅ Detailed Explanation    14
Network Connectivity                5/5 (100%)      ✅ Detailed Explanation    18
Compute System Components           1/5 (20%)       ✅ Detailed Explanation    5
Types of Compute Systems            3/4 (75%)       ✅ Detailed Explanation    4
Storage Architectures               2/5 (40%)       ✅ Detailed Explanation    3
Virtualization                      2/4 (50%)       ⚠️  Moderate Detail       2

📊 Overall Metrics:
----------------------------------------------

In [17]:
print("\n\n" + "=" * 100)
print("🏆 FINAL COMPARISON: Approach 3 vs Llama ")
print("=" * 100)

import pandas as pd

# Create comparison DataFrame with the metrics from your image
comparison_data = {
    'Metric': [
        'Topics Covered',
        'Topic Coverage %',
        'Avg Keyword Coverage',
        'Detail Score',
        'Detail Percentage',
        'Word Count',
        'Execution Time (s)',
        'Words per Second'
    ],
    'Approach 3 ': [
        f"{standalone_metrics['covered_topics']}/{standalone_metrics['total_topics']}",
        f"{standalone_metrics['covered_topics']/standalone_metrics['total_topics']*100:.1f}%",
        f"{standalone_metrics['avg_coverage']:.1f}%",
        f"{standalone_metrics['detail_score']:.1f}/{standalone_metrics['max_detail']}",
        f"{standalone_metrics['detail_percentage']:.1f}%",
        standalone_metrics['word_count'],
        f"{standalone_total_time:.2f}",
        f"{standalone_metrics['word_count']/standalone_total_time:.2f}"
    ],
    'Llama ': [
        f"{approach3_metrics['covered_topics']}/{approach3_metrics['total_topics']}",
        f"{approach3_metrics['covered_topics']/approach3_metrics['total_topics']*100:.1f}%",
        f"{approach3_metrics['avg_coverage']:.1f}%",
        f"{approach3_metrics['detail_score']:.1f}/{approach3_metrics['max_detail']}",
        f"{approach3_metrics['detail_percentage']:.1f}%",
        approach3_metrics['word_count'],
        f"{approach3_total_time:.2f}",
        f"{approach3_metrics['word_count']/approach3_total_time:.2f}"
    ],
    'Winner': []
}

# Determine winners
comparison_data['Winner'] = [
    '🏆 Llama' if approach3_metrics['covered_topics'] > standalone_metrics['covered_topics'] else '🏆 Approach 3',
    '🏆 Llama' if approach3_metrics['covered_topics'] > standalone_metrics['covered_topics'] else '🏆 Approach 3',
    '🏆 Llama' if approach3_metrics['avg_coverage'] > standalone_metrics['avg_coverage'] else '🏆 Approach 3',
    '🏆 Llama' if approach3_metrics['detail_score'] > standalone_metrics['detail_score'] else '🏆 Approach 3',
    '🏆 Llama' if approach3_metrics['detail_percentage'] > standalone_metrics['detail_percentage'] else '🏆 Approach 3',
    '🏆 Llama' if approach3_metrics['word_count'] > standalone_metrics['word_count'] else '🏆 Approach 3',
    '🏆 Approach 3' if standalone_total_time < approach3_total_time else '🏆 Llama',  # Faster is better
    '🏆 Llama' if approach3_metrics['word_count']/approach3_total_time > standalone_metrics['word_count']/standalone_total_time else '🏆 Approach 3'
]

df_comparison = pd.DataFrame(comparison_data)
print("\n")
print(df_comparison.to_string(index=False))

print("\n\n" + "=" * 100)
print("🎯 KEY INSIGHTS:")
print("=" * 100)

# Calculate differences
topic_diff = approach3_metrics['covered_topics'] - standalone_metrics['covered_topics']
coverage_diff = approach3_metrics['avg_coverage'] - standalone_metrics['avg_coverage']
detail_diff = approach3_metrics['detail_percentage'] - standalone_metrics['detail_percentage']
word_diff = approach3_metrics['word_count'] - standalone_metrics['word_count']
time_diff = approach3_total_time - standalone_total_time

print(f"\n1. 📊 TOPIC COVERAGE:")
if topic_diff > 0:
    print(f"   • Llama (Ensemble) covers {topic_diff} MORE topics than Standalone Llama")
    print(f"   • Llama (Ensemble) has {coverage_diff:.1f}% better keyword coverage across all topics")
else:
    print(f"   • Standalone Llama covers {abs(topic_diff)} MORE topics than Ensemble")
    print(f"   • Standalone Llama has {abs(coverage_diff):.1f}% better keyword coverage")

print(f"\n2. 📝 DETAIL LEVEL:")
if detail_diff > 0:
    print(f"   • Llama (Ensemble) provides {detail_diff:.1f}% more detailed explanations")
    print(f"   • T5 models extract diverse perspectives that Llama then expands")
else:
    print(f"   • Standalone Llama provides {abs(detail_diff):.1f}% more detailed explanations")
    print(f"   • Direct processing allows deeper focus on key concepts")

print(f"\n3. 📄 COMPREHENSIVENESS:")
if word_diff > 0:
    print(f"   • Llama (Ensemble) produces {word_diff} more words ({(word_diff/standalone_metrics['word_count']*100):.1f}% increase)")
    print(f"   • This is not just verbosity - it covers MORE topics with MORE detail")
else:
    print(f"   • Standalone Llama produces {abs(word_diff)} more words ({(abs(word_diff)/approach3_metrics['word_count']*100):.1f}% increase)")
    print(f"   • More concise while maintaining comprehensiveness")

print(f"\n4. ⏱️  TIME TRADE-OFF:")
if time_diff > 0:
    print(f"   • Llama (Ensemble) takes {time_diff:.1f} seconds longer ({(time_diff/standalone_total_time*100):.1f}% increase)")
    print(f"   • Extra time spent on T5 extraction: {t5_extraction_time:.2f}s")
    print(f"   • But produces {(approach3_metrics['word_count']/standalone_metrics['word_count']):.2f}x more comprehensive output")
else:
    print(f"   • Standalone Llama takes {abs(time_diff):.1f} seconds longer ({(abs(time_diff)/approach3_total_time*100):.1f}% increase)")
    print(f"   • Skipping T5 extraction saves time but may miss diverse perspectives")

print(f"\n5. 🎯 RECOMMENDATION:")
if approach3_metrics['covered_topics'] > standalone_metrics['covered_topics']:
    print(f"   ✅ CHOOSE LLAMA (Ensemble Method) because:")
    print(f"      • Covers {(approach3_metrics['covered_topics']/approach3_metrics['total_topics']*100):.0f}% of all topics vs {(standalone_metrics['covered_topics']/standalone_metrics['total_topics']*100):.0f}% for Standalone")
    print(f"      • Provides {detail_diff:.1f}% more detailed explanations")
    print(f"      • T5 models extract specialized perspectives (academic, news, conversational)")
    print(f"      • Worth the extra {time_diff:.1f}s for {abs(topic_diff)} more topics covered")
    print(f"      • Better for complex documents requiring comprehensive coverage")
else:
    print(f"   ✅ CHOOSE APPROACH 3 (Standalone Llama) because:")
    print(f"      • Faster by {abs(time_diff):.1f} seconds ({(abs(time_diff)/approach3_total_time*100):.1f}% reduction)")
    print(f"      • Covers {(standalone_metrics['covered_topics']/standalone_metrics['total_topics']*100):.0f}% of all topics")
    print(f"      • Simpler pipeline (single model)")
    print(f"      • Better for quick summaries or when T5 models unavailable")

print("\n" + "=" * 100)
print("📊 Analysis Complete!")
print("=" * 100)



🏆 FINAL COMPARISON: Approach 3 vs Llama 


              Metric Approach 3   Llama        Winner
      Topics Covered         8/8     6/8 🏆 Approach 3
    Topic Coverage %      100.0%   75.0% 🏆 Approach 3
Avg Keyword Coverage       68.1%   51.9% 🏆 Approach 3
        Detail Score     23.0/24 18.0/24 🏆 Approach 3
   Detail Percentage       95.8%   75.0% 🏆 Approach 3
          Word Count         935     722 🏆 Approach 3
  Execution Time (s)      103.68  111.44 🏆 Approach 3
    Words per Second        9.02    6.48 🏆 Approach 3


🎯 KEY INSIGHTS:

1. 📊 TOPIC COVERAGE:
   • Standalone Llama covers 2 MORE topics than Ensemble
   • Standalone Llama has 16.2% better keyword coverage

2. 📝 DETAIL LEVEL:
   • Standalone Llama provides 20.8% more detailed explanations
   • Direct processing allows deeper focus on key concepts

3. 📄 COMPREHENSIVENESS:
   • Standalone Llama produces 213 more words (29.5% increase)
   • More concise while maintaining comprehensiveness

4. ⏱️  TIME TRADE-OFF:
   • Ll